## Authors:

- Vinta Reethu       - ES18BTECH11028
- Chaitanya Janakie  - CS18BTECH11036
- Akash Tadwai       - ES18BTECH11019 

# CircularTrade Detection


In [8]:
import copy
import numpy as np
import pandas as pd
import pathlib

from typing import DefaultDict, List, Tuple, Set, Union 
from collections import defaultdict
from functools import reduce 

np.random.seed(42)

In [9]:
# Global Variables & Type Hints for the Notebook
"""
For testing the correctness of the algorithms, we have created the graph
which is given in the paper for which the values are already calculated.
we then applied the algorithms on this graph for which outputs we got are displayed

For implementing these algorithms on the large graph change the `path` variable to "PATH" 
while calling process_dataset function
"""

PATH = pathlib.Path('./cluster.csv')
test_dataset = pathlib.Path("./small_set.csv")
Graph = DefaultDict[int, List[Tuple[int, int]]]
Param = Tuple[int,int]
Vertex = int 
Cluster = defaultdict(set) 

In [10]:
def process_dataset(path: pathlib.Path):
    """Read and process dataset"""
    df = pd.read_csv(path)
    df.columns = ['v1', 'v2', 'amount']
    return df

df = process_dataset(test_dataset)
vertices = np.union1d(df['v1'],df['v2'])
graph = defaultdict(list)

for index, row in df.iterrows():
    v1, v2, amount = row
    graph[v1].append((v2, amount))

In [11]:
""" Util functions for implemeting further algorithms """

def reducetoSet(arr:set):
    return set(reduce(lambda a,b:set(a).union(set(b)),arr))

class StockFlowGraph:
    """Creates a stockflow graph from given graph and defines some common methods on stockflow graph"""

    def __init__(self,graph:Graph,k:int):
        self.graph = graph 
        self.k = k 
        self.mnvDict = defaultdict(dict) # stores (idx,weight) at which v2 appears in v1's kNN set 

    def _get_sfg(self, graph: Graph):
        """Return a Stock Flow Graph with max k neighbours"""
        sfg = copy.copy(graph)
        for v in vertices:
            sfg[v].sort(key=lambda x: x[1], reverse=True)
            # k-nearest neighbours of v
            topK = sfg[v][:self.k]

            for idx,ele in enumerate(topK):
                self.mnvDict[v][ele[0]]= (idx+1,ele[1])

            sfg[v] = set(map(lambda x: x[0],topK))
        return sfg 

    def getStockFlowGraph(self):
        # getter for stockflow graph 
        return self._get_sfg(self.graph) 

In [12]:
class SharedNN(StockFlowGraph):
    """ Implementation of SharedNN algorithm"""
    def __init__(self,k,kt):
        self.k=k
        self.kt=kt
        self.sfg = StockFlowGraph(graph,self.k)
    
    def toMerge(self,v:Vertex,u:Vertex,C:Cluster,s1:set,s2:set):
        # Condition to check to manipulate clusters 
        return (v not in C[u] and len(s1.intersection(s2))>=self.kt and u in s2 and v in s1)

    def fit(self,graph:Graph):
        # Implementation of the SharedNN algorithm on a graph 

        self._G = self.sfg.getStockFlowGraph()
        S = set(map(lambda x: tuple([x]),vertices))
        C = defaultdict(set) # C[x] contains all the vertices that are present in cluster where `x` belongs to.
        for key in vertices:
            C[key]|={key} #initially each vertex belongs to its own cluster

        for u in vertices:
            for v in vertices:
                s1 = self._G[u]
                s2 = self._G[v]
                if self.toMerge(v,u,C,s1,s2):
                    union = C[u].union(C[v])
                    S.remove(tuple(C[u]))
                    S.remove(tuple(C[v]))
                    S.add(tuple(union))
                    for ele in union:
                        C[ele]=union 
        filtered = list(filter(lambda x: len(x)>1,S)) # returns possible collusion sets
        return filtered

In [13]:
class MutualNNAvg(StockFlowGraph):
    """ Implementation of MutualNN algorithm"""
    def __init__(self,k,m):
        self.k=k
        self.m = m
        self.mxVal = 1000 # max value to break when mnvVal (mutual neighbourhood value) exceeds some limit.
        self.sfg = StockFlowGraph(graph,self.k)
        self.mnvDict = self.sfg.mnvDict 

    def mnvPoints(self,v1:Vertex,v2:Vertex):
        # Implementation of mutual neighbourhood value for two vertices in a graph 
        mnv, dist = (0,1)
        if v2 in self.mnvDict[v1]:
            mnv += self.mnvDict[v1][v2][0]
            dist = min(dist,-self.mnvDict[v1][v2][1])
        else:
            mnv += self.mxVal 
        if v1 in self.mnvDict[v2]:
            mnv += self.mnvDict[v2][v1][0]
            dist = min(dist,-self.mnvDict[v2][v1][1])
        else:
            mnv += self.mxVal
        
        return mnv, dist  

    def mnvClusters(self,c1:Cluster,c2:Cluster):
        """`mnv` value for two clusters """
        mnv, dist = (0, 1)
        for ele1 in c1:
            for ele2 in c2:
                val1, val2 = self.mnvPoints(ele1,ele2)
                mnv += val1
                dist = min(dist, val2)
        mnv = mnv/(len(c1)*len(c2))
        return mnv, dist 
        
    def fit(self, graph: Graph):
        # S contains collusion sets
        self._G = self.sfg.getStockFlowGraph()
        # Initially every vertex is a singleton cluster
        S = set(map(lambda x: tuple([x]),vertices))

        while len(S)>self.m:
            mnValue = float('inf')
            mnClusterdistance = 1
            clusterPair = None 
            for c1 in S:
                for c2 in S:
                    if c1==c2:
                        continue 
                    mnv, dist = self.mnvClusters(c1,c2)
                    if mnv<mnValue or (mnv==mnValue and mnClusterdistance>dist):
                        mnClusterdistance = dist
                        mnValue = mnv
                        clusterPair = (c1,c2)
                  
            if mnValue>=self.mxVal:
                break 
            S.remove(clusterPair[0])
            S.remove(clusterPair[1])
            S.add(tuple(set(clusterPair[0]).union(set(clusterPair[1]))))
        filtered = list(filter(lambda x: len(x)>1,S)) # returns possible collusion sets
        return filtered


In [14]:
def getCandidates(hyper_params:List[Param],algo:Union[SharedNN,MutualNNAvg]):
    # Given a list of hyperparams returns the possible candidate collusion set / fraud vertices 
    frauds = []
    for k,var in (hyper_params): # var can be either kt/m based on algorithm
        clusters = algo(k,var).fit(graph)
        print(f"\n Collusion set for k: {k}, kt/m: {var} \n ")
        print(sorted(clusters,key=lambda x: len(x),reverse=True))
        clusterSet = reducetoSet(clusters)
        frauds.append(clusterSet)
    print("\n Candidate Collusion set is: ")
    return set(reduce(lambda a,b:a.intersection(b),frauds))

In [15]:
# Testing for SharedNN Algorithm
hyper_params = [(5,2),(7,5),(6,4),(5,3),(4,2)]
print(getCandidates(hyper_params,SharedNN))


 Collusion set for k: 5, kt/m: 2 
 
[(1042, 1074, 1029, 1035, 1037)]

 Collusion set for k: 7, kt/m: 5 
 
[(1057, 1042, 1074, 1029, 1049, 1037)]

 Collusion set for k: 6, kt/m: 4 
 
[(1037, 1042, 1074, 1029)]

 Collusion set for k: 5, kt/m: 3 
 
[(1042, 1074, 1029, 1035, 1037)]

 Collusion set for k: 4, kt/m: 2 
 
[(1042, 1074, 1029, 1035, 1037)]

 Candidate Collusion set is: 
{1042, 1037, 1074, 1029}


In [16]:
# Testing on MutualNNAvg algorithm
hyper_params = [(2,1),(3,1),(4,1),(5,1)]
print(getCandidates(hyper_params,MutualNNAvg))


 Collusion set for k: 2, kt/m: 1 
 
[(1037, 1029, 1054)]

 Collusion set for k: 3, kt/m: 1 
 
[(1037, 1029, 1054), (1042, 1035)]

 Collusion set for k: 4, kt/m: 1 
 
[(1074, 1037, 1029, 1054), (1042, 1035)]

 Collusion set for k: 5, kt/m: 1 
 
[(1074, 1042, 1029, 1035, 1037, 1054)]

 Candidate Collusion set is: 
{1037, 1029, 1054}
